<a href="https://colab.research.google.com/github/vijayshankarrealdeal/intro_to_pytorch-Gans/blob/main/pytorch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
# !wget http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

In [64]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [65]:
df = pd.read_excel('/content/default of credit card clients.xls',skiprows=1)

In [66]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [67]:
data_clean = df.drop(['ID','SEX'],axis = 1)

In [68]:
data_clean.head()

,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [69]:
total = data_clean.isnull().sum()
percent = (data_clean.isnull().sum()\
           /data_clean.isnull().count()*100)
pd.concat([total, percent], axis=1, \
          keys=['Total', 'Percent']).transpose() 

,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
outliers = {}
for i in range(data_clean.shape[1]):
    min_t = data_clean[data_clean.columns[i]].mean() \
            - (3 * data_clean[data_clean.columns[i]].std())
    max_t = data_clean[data_clean.columns[i]].mean() \
            + (3 * data_clean[data_clean.columns[i]].std())
    count = 0
    for j in data_clean[data_clean.columns[i]]:
        if j < min_t or j > max_t:
            count += 1
    percentage = count/data_clean.shape[0]
    outliers[data_clean.columns[i]] = "%.3f" % percentage
print(outliers)

{'LIMIT_BAL': '0.004', 'EDUCATION': '0.011', 'MARRIAGE': '0.000', 'AGE': '0.005', 'PAY_0': '0.005', 'PAY_2': '0.005', 'PAY_3': '0.005', 'PAY_4': '0.006', 'PAY_5': '0.005', 'PAY_6': '0.004', 'BILL_AMT1': '0.023', 'BILL_AMT2': '0.022', 'BILL_AMT3': '0.022', 'BILL_AMT4': '0.023', 'BILL_AMT5': '0.022', 'BILL_AMT6': '0.022', 'PAY_AMT1': '0.013', 'PAY_AMT2': '0.010', 'PAY_AMT3': '0.012', 'PAY_AMT4': '0.013', 'PAY_AMT5': '0.014', 'PAY_AMT6': '0.015', 'default payment next month': '0.000'}


In [71]:
target = data_clean["default payment next month"]
yes = target[target == 1].count()
no = target[target == 0].count()
print("yes %: " + str(yes/len(target)*100) + " - no %: " \
      + str(no/len(target)*100))

yes %: 22.12 - no %: 77.88000000000001


In [72]:
data_yes = data_clean[data_clean["default payment next month"] == 1]
data_no = data_clean[data_clean["default payment next month"] == 0]
over_sampling = data_yes.sample(no, replace=True, \
                                random_state = 0)
data_resampled = pd.concat([data_no, over_sampling], \
                            axis=0)

In [73]:
data_resampled = data_resampled.reset_index(drop=True)
X = data_resampled.drop(columns=["default payment next month"])
y = data_resampled ["default payment next month"]

In [74]:
X = (X - X.min())/(X.max() - X.min())
X.head()

,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,0.080808,0.333333,0.666667,0.224138,0.2,0.2,0.2,0.2,0.2,0.2,0.172392,0.079532,0.093789,0.173637,0.095470,0.272928,0.001738,0.000891,0.001116,0.001610,0.002345,0.009458
1,0.040404,0.333333,0.333333,0.275862,0.2,0.2,0.2,0.2,0.2,0.2,0.188100,0.111995,0.113407,0.186809,0.109363,0.283685,0.002290,0.001199,0.001339,0.001771,0.002506,0.001892
2,0.040404,0.333333,0.333333,0.620690,0.1,0.2,0.1,0.2,0.2,0.2,0.154144,0.071601,0.106020,0.179863,0.099633,0.275681,0.002290,0.021779,0.011160,0.014493,0.001615,0.001284
3,0.040404,0.166667,0.666667,0.275862,0.2,0.2,0.2,0.2,0.2,0.2,0.203506,0.120381,0.117974,0.178407,0.100102,0.276367,0.002862,0.001078,0.000733,0.001610,0.002345,0.001513
4,0.494949,0.166667,0.666667,0.137931,0.2,0.2,0.2,0.2,0.2,0.2,0.472126,0.457242,0.330672,0.671310,0.559578,0.625196,0.062961,0.023749,0.042409,0.032591,0.032237,0.026047


In [75]:
final_data = pd.concat([X, y], axis=1)
final_data.to_csv("dccc_prepared.csv", index=False)

In [76]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [77]:
X_test.shape

(9346, 22)

In [78]:
class Classifier(nn.Module):
  def __init__(self,input_units = 1,output_units = 1,hidden_units = 64):
    super(Classifier,self).__init__()
    self.gen = nn.Sequential(
        nn.Linear(input_units,hidden_units*2),
        nn.ReLU(),
        nn.Linear(hidden_units*2,hidden_units),
        nn.ReLU(),
        nn.Linear(hidden_units,output_units),
        nn.Sigmoid()
    )
  def forward(self,x):
    return self.gen(x)

In [79]:
epochs = 10
batch_size = 100
model = Classifier(X.shape[1],output_units = 1)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
loss_fn = nn.NLLLoss()

In [96]:
train_losses, dev_losses, train_acc, dev_acc= [], [], [], []
for e in range(epochs):
  running_loss = 0
  running_accu = 0
  iterations = 0
  for i in range(0,len(X),batch_size):
    iterations += 1
    b = i + batch_size
    X_batch = torch.tensor(X_train.iloc[i:b,:]. values).float()
    y_batch = torch.tensor(Y_train.iloc[i:b].values)
    pred = model(X_batch).squeeze()
    print(pred.shape)
    print(pred[:5])
    print(y_batch[:5])
    loss = loss_fn(pred,y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    ps = torch.exp(pred)
    top_p, top_class = ps.topk(1, dim=1)
    running_acc += accuracy_score(y_batch, top_class)
  dev_loss = 0
  acc = 0
  with torch.no_grad():
    pred_dev = model(X_dev_torch)
    dev_loss = criterion(pred_dev, y_dev_torch)
    ps_dev = torch.exp(pred_dev)
    top_p, top_class_dev = ps_dev.topk(1, dim=1)
    acc = accuracy_score(y_dev_torch, top_class_dev)
    train_losses.append(running_loss/iterations)
    dev_losses.append(dev_loss)
    train_acc.append(running_acc/iterations)
    dev_acc.append(acc)
    print("Epoch: {}/{}.. ".format(e+1, epochs), "Training Loss: {:.3f}.. ")

torch.Size([100])
tensor([0.5102, 0.5141, 0.5155, 0.5093, 0.5107], grad_fn=<SliceBackward>)
tensor([1, 0, 0, 1, 1])


ValueError: ignored